### 参考资料
- https://blog.csdn.net/andybegin/article/details/77884645
- http://python.jobbole.com/86481/
- http://python.jobbole.com/87310/
- https://segmentfault.com/a/1190000009781688

    
### 迭代器
- 可迭代的：可直接用于for循环的变量
    - 有关于它的包iterable 在collections里面
- 迭代器：满足可迭代的定义，而且还可以被next调用。list就是可迭代的但不是迭代器。
    - 有关于它的包iterator 也在collections里面
- 可以将可迭代的东西转化为，迭代器。
    - 用iter()方法

In [ ]:
#iter的用法。

### 生成器
- generator：一边循环一边计算下一个元素的算法或者机制
- 满足的条件：
    - 能生产出所需要的元素
    - 达到最后一个数后，就是越界的时候，能出现StopIteration异常
    - 能被next调用
- 生成器的写法
    - 函数中有yield
- 调用
    - 使用next调用

In [7]:
#常见的一个生成器就是range
for i in range(5):
    print(i)
    
# 这是采用列表生成器生成的一个列表
l = [i for i in range(2)]
print(type(l))
# 这就是一个生成器
g = (i for i in range(2))
print(type(g))

0
1
2
3
4
<class 'list'>
<class 'generator'>


In [3]:
# 这里给出正常函数和生成器的对比
# 正常函数

def pt():
    print("this is one")
    print("this is two")
    print("this is three")
    
    return None

pt()
    
# 生成器写法
print("*" *20)


def pt_1():
    print("this is one")
    yield 1
    print("this is two")
    yield 2
    print("this is three")
    yield 3

    
#生成一个生成器
g = pt_1()
one = next(g)
two = next(g)
three = next(g)

this is one
this is two
this is three
********************
this is one
this is two
this is three


In [ ]:
# 斐波那契数列用生成器实现

### 协程
- 协程呢，其实可以看成是一个生成器，因为里面有yield；但是真正的意思是这样的，就是相当于有个函数，我可以执行一会、暂停一会。执行和暂停都是我人为来控制。使用的就是yield的原理，执行到这相当于给个标记，下次从上次执行的后面执行。
- 协程里有两个重要的关键字
    - yield
    - send
    


In [9]:
#协程的一个小例子
def simple_coroutine():
    print('-> start')
    x = yield
    print('-> recived', x)

sc = simple_coroutine()
print(1111)
# 可以使用sc.send(None)，效果一样
next(sc)

print(2222)
sc.send('joseph')

1111
-> start
2222
-> recived joseph


StopIteration: 

#### 协程的终止
- 协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象）
- 终止协程的一种方式：
    - 发送某个哨符值，让协程退出。
    - 内置的 None 和Ellipsis 等常量经常用作哨符值。
- 异常
    - generator.throw(Exception)
    - generator.close()

In [10]:
# 异常和终止的例子
# 这里是让协程抛出指定异常，然后暂停协程。
# 在异常被处理过后，协程就会继续执行到下一个yield表达式
# 若果未处理，则向上冒泡，找到调用者去处理这个事
class DemoException(Exception):
    """
    就是一个普通的自定义异常
    custom exception
    """
    pass

def handle_exception():
    print('-> start')

    while True:
        try:
            x = yield
        except DemoException:
            print('-> run demo exception')
        else:
            print('-> recived x:', x)

    #因为前面有个while true 所以前面的代码会一直执行不会执行到这句话
    raise RuntimeError('this line should never run') 

# 实例化一个协程
he = handle_exception()
next(he) # 预激
he.send(10) # recived x: 10
he.send(20) # recived x: 20

he.throw(DemoException) # run demo exception

#可以看到异常被处理过后，协程继续向前调用
he.send(40) # recived x: 40
he.close()

-> start
-> recived x: 10
-> recived x: 20
-> run demo exception
-> recived x: 40


In [18]:
# 这是一个generator.close的例子
# 当生成器遇到generator.close()的时候会抛出GeneratorExit异常。
# 一般GeneratorExit没有值，None
# 如果生成器没有处理，或抛出StopIteration错误，调用方不会报错。
# 否则会向上冒泡
class DemoException(Exception):
    """
    custom exception
    """
    pass

def handle_exception():
    print('-> start')

    while True:
        try:
            x = yield
        except DemoException:
            print('-> run demo exception')
        else:
            print('-> recived x:', x)


he = handle_exception()
next(he)
he.send(10) # recived x: 10
he.send(20) # recived x: 20


try:
    he.send(40) # recived x: 40
    he.close()
    he.send(50) # recived x: 40
    he.close()
except Exception as e:
    # 从结果可以看到，GeneratorExit异常没有具体的值
    print(str(e))
    print(e.value)

-> start
-> recived x: 10
-> recived x: 20
-> recived x: 40

None


#### yield from
- 相当于是个通道，或者说是个中间商，使协程的调用更灵活。
- 主线程与协程之间的数据交互就通过它
- yield from会把StopIteration捕获，并将其作为yield from的返回值

In [34]:
# yield from的一个小例子
def gen():
    for c in 'AB':
        yield c

        
# 如果list里面放的是一个可迭代的内容话，则会生成一个列表，里面包含可迭代参数的所有内容

print (list(gen()))

def gen_1():
    yield from 'AB'
    
    
print(list(gen_1()))

A
['A', 'B']
['A', 'B']


#### 委派生成器
- 包含yield from表达式的生成器函数
- 就相当于一个中间商

In [ ]:
# 委派生成器的代码
from collections import namedtuple

# 定义一个命名元组
ResClass = namedtuple('Res', 'count average')


# 子生成器
# 子生成器的作用就是计算平均数
# 这个计算是这样的，每来一个数，计算一下平均数
# 当来的值时None的时候停止计算
# 返回一个（人数，平均数）的Res元组
def averager():
    total = 0.0
    count = 0
    average = None

    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


# 委派生成器
# 委派生成器的作用就相当于一个中间商
# 主程序要什么我就去向生成器要什么
# 从子生成器那边得到数据，然后组成一个字典
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()


# 客户端代码
def client():
    process_data = {
        'boys_1': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_2': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)

# run
client()

### asyncio
- 异步I/O
- 电脑的i/o操作十分的耗时，若果CPU每次都等i/o操作结果的话会十分的浪费资源。所以就找了一个中间的信筒一样的东西，CPU给指令过后，不等待结果，如果有结果了，就将结果放到信箱里面，然后CPU会定时去里面拿东西。
- 使用之前需要导入，import
    - import asyncio

In [1]:
# asyncio一个小例子
import asyncio

# 异步协程
@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    print("Start......")
    r = yield from asyncio.sleep(3)
    print("Done....")
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
#用完了得关闭
loop.close()

Hello world!
Start......
Done....
Hello again!


In [2]:
# 小案例
import threading
import asyncio

# 异步协程
@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    print('Start..... (%s)' % threading.currentThread())
    yield from asyncio.sleep(5)
    print('Done..... (%s)' % threading.currentThread())
    print('Hello again! (%s)' % threading.currentThread())

# 启动消息循环
loop = asyncio.get_event_loop()

# 运行两个task
tasks = [hello(), hello()]

# 运行到完成，异步等待任务完成
loop.run_until_complete(asyncio.wait(tasks))

# 完成过后然后关闭
loop.close()

RuntimeError: Event loop is closed

### async and await
- 用来替换上面的asyncio 和yield from
- 缺案例


### aiohttp
- 缺案例 去Linux的时候来补

### concurrent.futures
- 类似于其他语言的线程池的概念
- 利用mutilprocessing实现并行计算
- 以子进程的形式，并行运行多个Python解释器，利用CPU的多核来提高Python程序的运行速度。由于子进程与主解释器相分离，所以他们的全局解释器锁也是相互独立的。每个子进程都能够完整的使用一个CPU内核。

- concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
- submit(fn ,args,kwargs)
    - fn:异步执行的函数
    - 后面两个是参数

In [3]:
# 小例子
from concurrent.futures import ThreadPoolExecutor
import time


def return_future(msg):
    time.sleep(3)
    return msg


# 创建一个线程池
# 相当于有两个子线程
pool = ThreadPoolExecutor(max_workers=2)

# 往线程池加入2个task
f1 = pool.submit(return_future, 'hello')
f2 = pool.submit(return_future, 'world')

print(f1.done())
time.sleep(3)
print(f2.done())

print(f1.result())
print(f2.result())

False
False
hello
world
